# Table of Contents
 <p><div class="lev1"><a href="#Laplacian-Smoothing"><span class="toc-item-num">1&nbsp;&nbsp;</span>Laplacian Smoothing</a></div><div class="lev2"><a href="#Test---Query-Perfect-Storm"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Test - Query Perfect Storm</a></div>

# Laplacian Smoothing

## Test - Query Perfect Storm

In [6]:
movie = """
A PERFECT WORLD
MY PERFECT WOMAN
PRETTY WOMAN
"""

song = """
A PERFECT DAY
ELECTRIC STORM
ANOTHER RAIN DAY
"""

n_movie = movie.count('\n')-1
n_song = song.count('\n')-1

p_movie = n_movie / (n_movie + n_song)
p_song = n_song / (n_movie + n_song)

(0.5, 0.5)